<a href="https://colab.research.google.com/github/yjc-spec/CNN-based-Emotion-Classification-with-Optimization/blob/main/Test_2270085_%EC%B5%9C%EC%97%B0%EC%9E%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, models

from torch.autograd import Variable
import torch.utils.data as data
import pandas as pd
import os
from PIL import Image
from torch.utils.data import DataLoader, TensorDataset, Dataset

import sklearn.model_selection as sk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path for (unlabeled) test data
test_path = '/content/drive/MyDrive/TestData_sample/'

In [ ]:
# Prepare test data set
# IMPORTANT: pre-processing should be same as in training

class FacialDataset(Dataset):
    def __init__(self, file_paths, labels=None, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img = Image.open(self.file_paths[idx]).convert('RGB')
        if self.transform:
            img = self.transform(img)
        else:
            img = transforms.ToTensor()(img)
        if self.labels is not None:
            return img, torch.tensor(self.labels[idx], dtype=torch.long)
        else:
            return img

test_file_paths = sorted([os.path.join(test_path, f) for f in os.listdir(test_path)])

test_dataset = FacialDataset(test_file_paths, labels=None)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

In [ ]:
# Build convolutional neural net
# IMPORTANT: This structure should be same as the one used for training (so simply copy and paste the correponding codes)
##### To do #####

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3,16,3), nn.BatchNorm2d(16), nn.ReLU(),
            nn.Conv2d(16,16,3), nn.BatchNorm2d(16), nn.ReLU(),
            nn.Dropout2d(0.3), nn.MaxPool2d(3,2),

            nn.Conv2d(16,32,3), nn.BatchNorm2d(32), nn.ReLU(),
            nn.Conv2d(32,32,3), nn.BatchNorm2d(32), nn.ReLU(),
            nn.Dropout2d(0.3), nn.MaxPool2d(3,2),

            nn.Conv2d(32,64,3), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64,64,3), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Dropout2d(0.3), nn.MaxPool2d(3,2),

            nn.Conv2d(64,128,3), nn.BatchNorm2d(128), nn.ReLU(),
            nn.Conv2d(128,128,3), nn.BatchNorm2d(128), nn.ReLU(),
            nn.Dropout2d(0.3), nn.MaxPool2d(3,2),
        )
        self.fc1 = nn.Linear(128*5*5, 128)
        self.bn1 = nn.BatchNorm1d(128); self.relu1 = nn.ReLU(); self.drop1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128,64)
        self.bn2 = nn.BatchNorm1d(64);  self.relu2 = nn.ReLU(); self.drop2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(64,3)
    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.drop1(self.relu1(self.bn1(self.fc1(x))))
        x = self.drop2(self.relu2(self.bn2(self.fc2(x))))
        return self.fc3(x)

model = Net()
model.cuda()

Net(
  (conv): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Dropout2d(p=0.3, inplace=False)
    (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (9): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
    (12): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU()
    (14): Dropout2d(p=0.3, inplace=False)
    (15): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (16): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  

In [ ]:
# load trained model
##### To do #####

model = torch.load('/content/drive/MyDrive/Model_2270085_최연재', weights_only = False)

In [ ]:
# evaluation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()
f_pred = []

with torch.no_grad():
    for inputs in test_loader:
        inputs = inputs.to(device)
        output = model(inputs)
        _, pred = output.max(1)
        f_pred.extend(pred.detach().cpu().numpy().tolist())

In [ ]:
# Submission file
img_files = [i+1 for i in range(len(f_pred))]
submission = pd.DataFrame(index = range(0,len(f_pred)), columns = ['img','pred_class'])
submission['img'] = img_files
submission['pred_class'] = f_pred

In [ ]:
submission

,img,pred_class
0,1,0
1,2,1
2,3,2
3,4,0
4,5,1
5,6,0
6,7,1
7,8,2
8,9,2
9,10,2
